# 2 - How the backpropagation algorithm works 

In the last chapter we saw how neural networks can learn their weights and biases using the gradient descent algorithm. There was, however, a gap in our explanation: we didn't discuss how to compute the gradient of the cost function. In this chapter, we will learn a fast algorithm for computing such gradients, which known as $backpropagation$.

Backpropagation isn't just a fast alogrithm for learning. **It actually gives us detailed insights into how changing the weights and biases changes the overall behaviour of the network.** That's well worth study in detail. Let's enjoy it.

## 2.1 - Warm up: a fast matrix-based approach to compute the out from a neural network

Before discussing backpropagation, let's warm up with a fast matrix-based algorithm to compute the output from a neural network. In particular, this is a good way of getting comfortable with the notation used in backpropagation, in a familiar context. We acutally already saw this algorithm near the end of the last chapter (section 1.6), let's review it now.

We'll use $w^{l}_{jk}$ to denote the weight for the connection from the $k$-th neuron in the ($l$ - 1)-th layer to the $j$-th neuron in the $l$-th layer. So, for example, the diagram below shows the weight on a connection from the fourth neuron in the second layer to the second neuron in the third layer of a network:

![2.1](../picture/chapter2/p1.png)

We use a similar notation for the network's biases and activations. Explicitly, we use $b^{l}_{j}$ for the bias of the $j$-th neuron in the $l$-th layer. And we use $a^{l}_{j}$ for the activation of the $j$-th neuron in the $l$-th layer. The following diagram shows examples of these notations in use:

![2.1](../picture/chapter2/p2.png)

With these notations, **the activation $a^{l}_{j}$ of the $j$-th neuron in the $l$-th layer is related to the activations in the ($l$ - 1)-th layer by the equation**

$$a^{l}_{j} = \sigma(\sum_{k} w^{l}_{jk}a^{l-1}_{k} + b^{l}_{j}) , \tag{2.1}$$

where the sum is over all neurons $k$ in the ($l$ - 1)-th layer.

To rewrite this expression in a matrix form, we define a weight maxtrix $w^l$ for each layer, $l$. Similarly, for each layer $l$ we define a bias vector, $b^l$. And finally, we define an activation vector $a^l$ whose components are the activations $a^{l}_{j}$. The last ingredient we need to rewrite $2.1$ in a matrix form is the idea of vectorizing a function such as $\sigma$. The idea is that we want to apply a function such as $\sigma$ to every element in a vector $v$.

With these notations in mind, Equation 2.1 can be rewritten in the beautiful and compact vectorized form

$$a^{l} = \sigma(w^{l}a^{l-1} + b^{l}). \tag{2.2}$$

**This expression gives us a much more global way of thinking about how the activations in one layer relate to activations in the previous layer: we just apply the weight matrix to the activations, then add the bias vector, and finally apply the $\sigma$ function.**

When using Equation 2.2 to compute $a^l$, we compute the intermediate quantity $z^{l} \equiv w^la^{l-1} + b^l$ along the way. **This quantity turns out to be useful enough to be worth naming: we call $z^l$ the weighted input to the neurons in layer $l$. We'll make considerable use of the weighted input $z^l$ later in the chapter.**

## 2.2 - The two assumptions we need about the cost function

The goal of backpropagation is to compute the partial derivatives $\partial C/ \partial w$ and $\partial C/ \partial b$ of the cost function $C$ with respect to any weight $w$ or bias $b$ in the network. **For backpropagation to work we need to make two main assumptions about the form of the cost function.** We use the quadratic cost function as a example to state those assumptions. As we learned from last chapter, the quadratic cost has the form

$$C = \frac{1}{2n}\sum_x \|y(x) - a^L(x)\|^2 , \tag{2.3}$$

where L denotes the number of layers in the network and $a^L = a^L(x)$ is the vector of activations output from the network when x is input.

The first assumption we need is that the cost function can be written as an average $C = \frac{1}{n}\sum_x C_x$ over cost functions $C_x$ for individual training examples, x. This is the case for the quadratic cost function, where the cost for a single training example is $C_x = \frac{1}{2}\|y - a^L\|^2$. **The reason we need this assumption is beacuase we then recover $\partial C/\partial w$ and $\partial C/\partial b$ by averaging over training examples.**

The second assumption we make about the cost is that is can be written as a function of the outputs from the neural network:

![2.2](../picture/chapter2/p3.png)

For example, the quadratic cost function satisfies this requirement, since the quadratic cost for a sigle training example $x$ may be written as(the following $y$ and $a^L$ are both vector)

$$C = \frac{1}{2}\|y - a^L\|^2 = \frac{1}{2}\sum_{j}(y_j - a^{L}_{j})^2, \tag{2.4}$$

and thus is a function of the activations output.

## 2.3 - The Hadamard product, $s \odot t$

Suppose $s$ and $t$ are two vectors of the same dimension. Then we use $s \odot t$ to denote the elementwise product of the two vectors. As an example,

$$ \begin{bmatrix} 1 \\ 2 \end{bmatrix} \odot \begin{bmatrix} 3 \\ 4 \end{bmatrix} = \begin{bmatrix} 1*3 \\ 2*4 \end{bmatrix} = \begin{bmatrix} 3 \\ 8 \end{bmatrix}. \tag{2.5}$$

This kind of elementwise multiplication is sometimes called the $Hadamard\;product$ or $Schur\;product$. We'll refer to it as the Hadamard product.

## 2.4 - The four fundamental equations behind backpropagation

We first introduce an intermediate quantity, $\delta^l_j$, which we call the $error$ in the $j$-th neuron in the $l$-th layer. Backpropagation will give us a procedure to compute the error $\delta^l_j$, and then relate $\delta^l_j$ to $\partial C/\partial w^{l}_{jk}$ and $\partial C / \partial b^{l}_{j}$. We define the error $\delta^l_j$ of neuron $j$ in layer $l$ by

$$\delta^l_j \equiv  \frac{\partial C}{\partial z^l_j} .$$

**(What does the term of $\delta^{l}_{j}$ mean? I can't fully understand the explanation in this book. Now, I just consider it as an intermediate quantity.)**

**I may appreciate this beautity of such design through the process of proof the four equations. It's really a ingenious intermediate quantity, which relates to $\partial C/\partial w^{l}_{jk}$ and $\partial C / \partial b^{l}_{j}$ and makes calculation of gradient easily.**

### Equation Ⅰ
**An equation for the error in the output layer,** $\delta^L$: The components of $\delta^L$ are given by 

$$\delta^L_j = \frac{\partial C}{\partial a^L_j}\sigma '(z^L_j) \tag{BP1}$$

**The first term on the right, $\partial C / \partial a^L_j$, just measures how fast the cost is changing as a function of the $j$-th output activation. If, for example, $C$ doesn't depend much on a particular output neuron, $j$, then $\delta^L_j$ will be small, which is what we'd expect. The second term on the right, $\sigma '(z^L_j)$, measures how fast the activation function $\sigma$ is changing at $z^L_j$.**

Notice that everything in Eq. BP1 is easily computed. In particular, we compute $z^L_j$ while computing the behaviour of the network. The exact form of $\partial C / \partial a^L_j$ will, of course, depend on the form of the cost function. For example, if we're using the quadratic cost function then $C = \frac{1}{2}\sum_{j}(y_j - a^L_j)^2$, and so $\partial C / \partial a^L_j = (a^L_j - y_j)$.(the second assumption)

It's easy to rewrite the equation in a matrix-based form, as

$$\delta^L = \triangledown_aC\odot\sigma'(z^L). \tag{BP1a}$$

Here, $\triangledown_aC$ is defined to be a vector whose components are the partial derivatives $\partial C / \partial a^L_j$. **You can think of $\triangledown_aC$ as expressing the rate of change of $C$ with respect to the output activations.** As an example, in the case of the quadratic cost we have $\triangledown_aC = (a^L - y)$, and so the fully matrix-based form of BP1 becomes

$$\delta^L = (a^L - y) \odot \sigma'(z^L). \tag{2.6}$$

### Equation Ⅱ

**An equation for the error $\delta^l$ in terms of the error in the next layer**, $\delta^{l+1}$: In particular

$$\delta^l = ((w^{l+1})^T\delta^{l+1})\odot \sigma'(z^l), \tag{BP2}$$


where $(w^{l+1})^T$ is the transpose of the weight matrix $w^{l+1}$ for the ($l$+1)-th layer.

**By combining (BP2) with (BP1) we can compute the error $\delta^l$ for any layer in the network. We start by using (BP1) to compute $\delta^L$, then apply Equation (BP2) again to compute $\delta^{L-1}$, then Equation (BP2) again to compute $\delta^{L-2}$, and so on, all the way back through the network.**

### Equation Ⅲ

**An equation for the rate of change of the cost with respect to any bias in the network:** In particular:

$$\frac{\partial C}{\partial b^l_j} = \delta^l_j. \tag{BP3}$$

That is, the error $\delta^l_j$ is exactly equal to the rate of change $\partial C / \partial b^l_j$.

### Equation Ⅳ

**An equation for the rate of change of the cost with respect to any weight in the network:** particular:

$$\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k\delta^l_j. \tag{BP4}$$

The euation can be rewritten in a less index-heavy notation as

$$\frac{\partial C}{\partial w} = a_{in}\delta_{out}, \tag{2.7}$$

where it's understood that $a_{in}$ is the activation of the neuron input to the weight $w$, and $\delta_{out}$ is the error of the neuron output from the weight $w$.

**A nice consequence of Equation 2.7 is that when the activation $a_{in}$ is small, $a_{in} \approx 0$, the gradient term $\partial C / \partial w$ will also tend to be small. In this case, we'll say the weight $learns$ $slowly$, meaning that it's not changing much during gradient descent. In other words, one consequence of (BP4) is that weights output from low-activation neurons learn slowly.**

**There are other insights along these lines which can be obtained from (BP1)-(BP4). Let' start by looking at the output layer. Consider the term $\sigma'(z^L_j)$ in (BP1). Recall from the graph of the sigmoid function in the last chapter that the $\sigma$ function becomes very flat when $\sigma(z^L_j)$ is approximately 0 or 1. When this occurs we will have $\sigma'(z^L_j) \approx 0$. And so the lesson is that a weight in the final layer will learn slowly if the output neuron is either low activation ($\approx 0$) or high activation ($\approx 1$). In this case it's common to say the output neuron has $saturated$.**

**We can obtain similar insights for earlier layers. In particular, note the $\sigma'(z^l)$ term in (BP2). This means that $\delta^l_j$ is like to get small if the neuron is near saturation.**

**Summing up, we've learnt that a weight will learn slowly if either the input neuron is low-activation($a^{l-1}_k\approx0$),or if the output neuron has saturated, i.e., is either high- or low-activation.**

**We can also use these insights to design activation functions which have particular desired learning properties.** Later in the book we'll see examples where this kind of modification is made to the activation function. Keeping the four equations BP1-BP4 in mind can help explain why such modifications are tried, and what impact they can have.

### Summary: the equations of backpropagation

$$\delta^L = \triangledown_aC\odot\sigma'(z^L) \tag{BP1}$$

$$\delta^l = ((w^{l+1})^T\delta^{l+1})\odot\sigma'(z^l) \tag{BP2}$$

$$\frac{\partial C}{\partial b^l_j} = \delta^l_j \tag{BP3}$$

$$\frac{\partial C}{\partial w^{l}_{jk}} = a^{l-1}_{k}\delta^l_j \tag{BP4}$$

## 2.5 - Proof of the four fundamental equations

We'll now prove the four fundamental equations (BP1)-(BP4). **All four are consequences of the chain rule from multivariable calculus.**

Let's begin with Equation (BP1), which gives an expression for the output error, $\delta^L$. To prove this equation, recall that by definition

$$\delta^L_j \equiv  \frac{\partial C}{\partial z^L_j} .\tag{2.8}$$

Applying the chain rule, we can re-express the partial derivative above in terms of partial derivatives with respect to the output activations,

$$\delta^L_j = \sum_{k}\frac{\partial C}{\partial a^L_k}\frac{\partial a^L_k}{\partial z^L_j}, \tag{2.9}$$

where the sum is over all neurons $k$ in the output layer. Of course, the output activation $a^L_k$ of the $k$-th neuron depends only on the weighted input $z^L_j$ for the $j$-th neuron when $k$ = $j$. And so $\partial a^L_k / \partial z^L_j$ vanishes when $k\ne j$. As a result we can simplify the previous equation to

$$\delta^L_j = \frac{\partial C}{\partial a^L_j}\frac{\partial a^L_j}{\partial z^L_j}. \tag{2.10}$$

Recalling that $a^L_j = \sigma(z^L_j)$ the second term on the right can be rewritten as $\sigma'(z^L_j)$, and the equation becomes

$$\delta^L_j = \frac{\partial C}{\partial a^L_j}\sigma'(z^L_j), \tag{2.11}$$

which is just (BP1), in component form. Next, we'll prove (BP2), which gives an equation for the error $\delta^l$ in terms of the error in the next layer, $\delta^{l+1}$. To do this, we want to rewrite $\delta^l_j = \partial C / \partial z^{l}_j$ in terms of $\delta^{l+1}_k = \partial C / \partial z^{l+1}_k$. We can do this using the chain rule,

$$\delta^l_j = \frac{\partial C}{\partial z^l_j} \stackrel{?}= \sum_{k}\frac{\partial C}{\partial z^{l+1}_k}\frac{\partial z^{l+1}_k}{\partial z^l_j} = \sum_{k}\frac{\partial z^{l+1}_k}{\partial z^l_j}\delta^{l+1}_k . \tag{2.12}$$
 

**I can't understand the equation which I had sign with symbol of ?.**

To evaluate the first term on the last line, note that

$$z^{l+1}_k = \sum_{j}w^{l+1}_{kj}a^{l}_{j} + b^{l+1}_{k} = \sum_{j}w^{l+1}_{kj}\sigma(z^l_j) + b^{l+1}_{k}. \tag{2.13}$$

Differentiating, we obtain

$$\frac{\partial z^{l+1}_k}{\partial z^l_j} = w^{l+1}_{kj}\sigma'(z^l_j). \tag{2.14}$$

Substituting back into (2.12) we obtain

$$\delta^l_j = \sum_{k}w^{l+1}_{kj}\delta^{l+1}_k\sigma'(z^{l}_j). \tag{2.15}$$

This is just (BP2) written in component form.

## 2.6 - The backpropagation algorithm

The backpropagation equations provide us with a way of computing the gradient of the cost function. Let's explicitly write this out in the form of an algorithm:

1. **Input** $x$: Set the corresponding activation $a^1$ for the input layer.
2. **Feedforward**: For each $l$ = 2, 3,..., L compute $z^l = w^la^{l-1} + b^l$ and $a^l = \sigma(z^l)$.
3. **Output error** $\delta^L$: Compute the vector $\delta^L = \triangledown_aC\odot\sigma'(z^L).$
4. **Backpropagate the error**: For each $l$ = L-1, L-2,..., 2 compute $\delta^l = ((w^{l+1})^T\delta^{l+1})\odot\sigma'(z^l)$.
5. **Output**: The gradient of the cost function is given by $\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k\delta^l_j$ and $\frac{\partial C}{\partial b^l_j} = \delta^l_j$.

Examining the algorithm you can see why it's called $backpropagation$. We compute the error vectors $\delta^l$ backward, starting from the final layer. It may seem peculiar that we're going through the network backward. **But if you think about the proof of backpropagation, the backward movement is a consequence of the fact that the cost is a function of outputs from the network.**

**In practice, it's common to combine backpropagation with a learning algorithm such as stochastic gradient descent, in which we compute the gradient for many training examples.** In particular, given a mini-batch of $m$ training examples, the following algorithm applies a gradient descent learning step based on that mini-batch:


1. Input a set of training examples.
2. For each training example $x$: Set the corresponding input activation $a^{x,1}$, and perform the following steps:

  * Feedforward: For each $l$= 2, 3,..., L compute $z^{x,l} = w^la^{x,l-1} + b^l$ and $a^{x, l} = \sigma(z^{x,l}).$
  * Output error $\delta^{x,L}$: Compute the vector $\delta^{x,L} = \triangledown_aC_x\odot\sigma'(z^{x,L}).$
  * Backpropagate the error: For each $l$ = L - 1, L - 2,..., 2 compute $\delta^{x,l} = ((w^{l+1})^T\delta^{x,l+1})\odot\sigma'(z^{x,l}).$
3. Gradient descent: For each $l$ = L, L-1,...,2 update the weights according to the rule $w^l \rightarrow w^l - \frac{\eta}{m}\sum_x\delta^{x,l}(a^{x,l-1})^T$, and the biases according to the rule $b^l\rightarrow b^l - \frac{\eta}{m}\sum_x\delta^{x,l}.$

## 2.7 - The code for backpropagation

Having understood backpropagation in the abstract, we can now understand the code used in the last chapter to implement backpropagation. The code is a direct translation of the algorithm described above. **In particular, the code updates the Network's weights and biases by computing the gradient for the current mini-batch of training examples.**

## 2.8 - In what sense is backpropagation a fast algorithm?

In what sense is backpropagation a fast algorithm? To answer this question, let's consider another approach to compute the gradient. An obvious approach is to use the approximation

$$\frac{\partial C}{\partial w_j} \approx \frac{C(w + \epsilon e_j) - C(w)}{\epsilon} \tag{2.26},$$

where $\epsilon > 0$ is a small positive number, and $e_j$ is the unit vector in the $j$-th direction. In other words, we can estimate $\partial C / \partial w_j$ by computing the cost $C$ for two slightly different values of $w_j$, and then applying Equation 2.26. The same idea will let us compute the partial derivatives $\partial C / \partial b$ with respect to the biases.

**This approach looks very promising, but imagine we have a million weights in our network. Then for each distinct weight $w_j$ we need to compute $C(w + \epsilon e_j)$ in order to compute $\partial C / \partial w_j$. That means that to compute the gradient we need to compute the cost function a million different times, requiring a million forward passes through the network(per training example). We need to compute $C(w)$ as well, so that's a total of a million and one  passes through the network.**

**What's clever about backpropagation is that it enables us to simultaneously compute all the partial derivatives $\partial C / \partial w_j$ using just one forward pass through the network, followed by one backward pass through the network. Roughly speaking, the computational cost of the backward pass is about the same as the forward pass. And so the total cost of backpropagation is roughly the same as making just two forward passes through the network.**

**Of course, backpropagation is not a panacea. Even in the late 1980s people ran up against limits, especially when attempting to use backpropagation to train deep neural networks. Later in the book, we'll see how modern computers and some clever new ideas now make it possible to use backpropagation to train such deep neural networks.**

## 2.9 - Backpropagation: the big picture

To improve our intuition about what the algorithm is doing, let's imagine that we've made a small change $\Delta w^l_{jk}$ to some weight in the network, $w^l_{jk}$. The change $\Delta C$ in the cost is related to the change $\Delta w^l_{jk}$ in the weight by the equation

$$\Delta C \approx \frac{\partial C}{\partial w^l_{jk}}\Delta w^l_{jk} \tag{2.27}$$

This suggests that a possible approach to computing $\partial C / \partial w^l_{jk}$ is to carefully track how a small change in $w^l_{jk}$ propagates to cause a small change in C. We can do this by using chain rule. If the path goes through activations $a^l_j, a^{l+1}_q,..., a^{L-1}_{n}, a^L_m$ then the resulting expression is

$$\Delta C \approx \frac{\partial C}{\partial a^L_m} \frac{\partial a^L_m}{\partial a^{L-1}_n} \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p}...\frac{\partial a^{l+1}}{\partial a^l_j} \frac{\partial a^l_j}{\partial w^l_{jk}}\Delta w^l_{jk} \tag{2.28}.$$

To compute the total change in $C$ it is plausible that we should sum over all possible paths between the weight and the final cost, i.e.,

$$\Delta C \approx \sum_{mnp...q}\frac{\partial C}{\partial a^L_m} \frac{\partial a^L_m}{\partial a^{L-1}_n} \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p}...\frac{\partial a^{l+1}}{\partial a^l_j} \frac{\partial a^l_j}{\partial w^l_{jk}}\Delta w^l_{jk} \tag{2.29}.$$

Comparing with (2.27) we see that

$$\frac{\partial C}{\partial w^l_{jk}} =  \sum_{mnp...q}\frac{\partial C}{\partial a^L_m} \frac{\partial a^L_m}{\partial a^{L-1}_n} \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p}...\frac{\partial a^{l+1}}{\partial a^l_j} \frac{\partial a^l_j}{\partial w^l_{jk}}\Delta w^l_{jk} \tag{2.30}.$$